In [1]:
from raster_to_tile import data_split

tif_selva = r"D:\Shapefiles\PI\CO_2512022106159\VOL_PER1_ORT_001_010438\IMG_PER1_ORT_PMS_010438\IMG_PER1_20230922151914_ORT_PMS_010438.TIF"
tif_sierra = r"D:\Shapefiles\PI\CO_2510221339013\VOL_PER1_ORT_001_002705\IMG_PER1_ORT_PMS_002705\IMG_PER1_20210706153514_ORT_PMS_002705.TIF"

mask_casas_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\buildings_32718_selva.gpkg"
mask_casas_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\buildings_32717_sierra.gpkg"

mask_caminos_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\capa_carreteras_selva.gpkg"
mask_caminos_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\\capa_carreteras_sierra.gpkg"

gpkg_division_selva_tile = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\division_en_tile_selva.gpkg"
gpkg_division_sierra_tile = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\division_en_tile_sierra.gpkg"

dataset_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSelvaMulticlase"
dataset_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSierraMulticlase1"

### Area Excluida
gpkg_area_ciudad = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\area_excluida_sierra.gpkg"

data_split(tif_sierra, mask_casas_sierra_path, mask_caminos_sierra_path, gpkg_division_sierra_tile, dataset_sierra_path, gpkg_area_excluida_path=gpkg_area_ciudad)

Leyendo raster entero para calcular percentiles para posterior normalización de parches
Cargando raster...
Cargando geometrías...
Geometria de : Casas
Geometria de : Caminos
Geometria de : Area fuera de interes
Cargando tiles...

Procesando tile 0 → train

Procesando tile 1 → test

Procesando tile 2 → train

Procesando tile 3 → val

Procesando tile 4 → train

Procesando tile 5 → val

Procesando tile 6 → train

Procesando tile 7 → train

Procesando tile 8 → train

Procesando tile 9 → train

Procesando tile 10 → train

Procesando tile 11 → test

Procesando tile 12 → train
Se excluyo el tile: 3064
Se excluyo el tile: 3081
Se excluyo el tile: 3081
Se excluyo el tile: 3098
Se excluyo el tile: 3098
Se excluyo el tile: 3115
Se excluyo el tile: 3115
Se excluyo el tile: 3132
Se excluyo el tile: 3132
Se excluyo el tile: 3149
Se excluyo el tile: 3149
Se excluyo el tile: 3166
Se excluyo el tile: 3166
Se excluyo el tile: 3183
Se excluyo el tile: 3183
Se excluyo el tile: 3200
Se excluyo el tile: 320

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print("GPUs disponibles:", gpus)

GPUs disponibles: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [1]:
from utils2 import split_dataset
dataset_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSierraMulticlase"

(train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks) = split_dataset(carpeta=dataset_path)


División correcta sin errores de archivos.
--------------------------------------------------
TRAIN     → imágenes: 4117 | máscaras: 4117
VALIDATION → imágenes: 938 | máscaras: 938
TEST      → imágenes: 969 | máscaras: 969
--------------------------------------------------
TOTAL IMGS: 6024
TOTAL MASKS: 6024
--------------------------------------------------


In [2]:
from TiffDataGeneratorAugmentedUnet import TIFFDataGeneratorAug

# Generadores
train_gen = TIFFDataGeneratorAug(train_imgs, train_masks, normalize='imagenet',batch_size=2, n_channels=3, augment=True)#<--- canales
val_gen = TIFFDataGeneratorAug(val_imgs, val_masks, normalize='imagenet', batch_size=2, shuffle=False, n_channels=3, augment=False)#<--- canales
test_gen = TIFFDataGeneratorAug(test_imgs, test_masks, normalize='imagenet', batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales


c:\Users\prac-dnce\miniconda3\envs\tensorflow\lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
d:\Diciembre\Entrenamiento\Unet\TiffDataGeneratorAugmentedUnet.py:49: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(3, 10), p=0.1)


In [3]:
x, y = train_gen[0]
print(x.shape, y.shape)

(2, 512, 512, 3)
(2, 512, 512, 3) (2, 512, 512, 1)


In [4]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [5]:
def masked_sparse_cce(y_true, y_pred):
    y_true = tf.squeeze(y_true, axis=-1)

    mask = tf.not_equal(y_true, 255)
    y_true = tf.boolean_mask(y_true, mask)
    y_pred = tf.boolean_mask(y_pred, mask)

    return tf.keras.losses.sparse_categorical_crossentropy(
        y_true, y_pred
    )

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

BACKBONE = 'resnet34'#efficientnetb0 tmb puede ser (buena con imag satelitales)
model = sm.Unet(
    backbone_name=BACKBONE,
    encoder_weights='imagenet',
    classes=3,
    activation='softmax'
    #input_shape=(512, 512, 4)
)

model.compile(
    optimizer=Adam(1e-4),
    loss=masked_sparse_cce,
    metrics=[tf.keras.metrics.MeanIoU(num_classes=3, name="iou"), 
             "accuracy"]
)

In [9]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   9           ['data[0][0]']                   
                                3)                                                                
                                                                                                  
 zero_padding2d_34 (ZeroPadding  (None, None, None,   0          ['bn_data[0][0]']                
 2D)                            3)                                                          

In [26]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Early stopping: detener cuando el modelo deje de mejorar
early_stop = EarlyStopping(
    monitor='val_iou',      # métrica que se evalúa
    mode='max',           # porque cuanto más alto, mejor
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Guardar el mejor modelo automáticamente
checkpoint = ModelCheckpoint(
    r"D:\Diciembre\Entrenamiento\training_road_model\Sierra\Checkpoints\checkpoint.keras",         # ruta del archivo
    monitor='val_iou',
    save_best_only=True,
    verbose=1
)


In [27]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    verbose=1,
    callbacks=[early_stop,checkpoint]
)
print("Epochs realizadas:", len(history.history['loss']))

(2, 512, 512, 3)
Epoch 1/100
(2, 512, 512, 3)
(2, 512, 512, 3)


: 

In [ ]:
model.save(r"D:\Diciembre\Entrenamiento\training_road_model\Checkpoints\model.keras",)

In [ ]:
model.save_weights(r"D:\Diciembre\Entrenamiento\training_road_model\Checkpoints\model.weights.h5")

In [ ]:
train_loss, train_iou, train_f1 = model.evaluate(train_gen, verbose=1)
val_loss, val_iou, val_f1 = model.evaluate(val_gen, verbose=1)
test_loss, test_iou, test_f1 = model.evaluate(test_gen, verbose=1)


print("Resultados en TRAIN:")
print("  Loss:", train_loss)
print("  IoU :", train_iou)
print("  F1  :", train_f1)

print("Resultados en VAL:")
print("  Loss:", val_loss)
print("  IoU :", val_iou)
print("  F1  :", val_f1)

print("Resultados en TEST:")
print("  Loss:", test_loss)
print("  IoU :", test_iou)
print("  F1  :", test_f1)
